# Batching all waves 

### Subjects with all waves 1-4, separated into 2 parts:
#### All subjects with 4 waves (1st half):  
['sub-005',
 'sub-011',
 'sub-012',
 'sub-022',
 'sub-030',
 'sub-032',
 'sub-037',
 'sub-040',
 'sub-041',
 'sub-042',
 'sub-043',
 'sub-047',
 'sub-052',
 'sub-054',
 'sub-055',
 'sub-056',
 'sub-059',
 'sub-063',
 'sub-065',
 'sub-068',
 'sub-076',
 'sub-079',
 'sub-081',
 'sub-082',
 'sub-084',
 'sub-085',
 'sub-093',
 'sub-097',
 'sub-098',
 'sub-099',
 'sub-101',
 'sub-102',
 'sub-104',
 'sub-106',
 'sub-113',
 'sub-114',
 'sub-115',
 'sub-117',
 'sub-120',
 'sub-122',
 'sub-130']  
   
#### Test Subjects | 20% randomly selected:  
['sub-030',
 'sub-047',
 'sub-055',
 'sub-059',
 'sub-079',
 'sub-097',
 'sub-101',
 'sub-117']
 
 
   
   #### All subjects with 4 waves (2nd half):  
['sub-001',
 'sub-004',
 'sub-013',
 'sub-028',
 'sub-029',
 'sub-033',
 'sub-034',
 'sub-044',
 'sub-045',
 'sub-046',
 'sub-048',
 'sub-049',
 'sub-051',
 'sub-053',
 'sub-057',
 'sub-058',
 'sub-061',
 'sub-064',
 'sub-066',
 'sub-071',
 'sub-075',
 'sub-077',
 'sub-083',
 'sub-088',
 'sub-089',
 'sub-092',
 'sub-094',
 'sub-095',
 'sub-096',
 'sub-103',
 'sub-105',
 'sub-107',
 'sub-109',
 'sub-110',
 'sub-116',
 'sub-118',
 'sub-119',
 'sub-123',
 'sub-126',
 'sub-128',
 'sub-131',
 'sub-154'] 
   
#### Test Subjects | 20% randomly selected:   
['sub-001',
 'sub-048',
 'sub-049',
 'sub-058',
 'sub-064',
 'sub-105',
 'sub-107',
 'sub-116']



In [ ]:
import os
import glob
import numpy as np
import nibabel as nib
from random import shuffle
import pandas as pd


wave4 = sorted(glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/sub-*/ses-4/func/Ana*/feat1/task-*/filtered_func_data.nii.gz")))
wave3 = sorted(glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/sub-*/ses-3/func/Ana*/feat1/task-*/filtered_func_data.nii.gz")))
wave2 = sorted(glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/sub-*/ses-2/func/Ana*/feat1/task-*/filtered_func_data.nii.gz")))
wave1 = sorted(glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/sub-*/ses-1/func/Ana*/feat1/task-*/filtered_func_data.nii.gz")))

subj4 = [i.split("/")[-7] for i in wave4]
subj3 = [i.split("/")[-7] for i in wave3]
subj2 = [i.split("/")[-7] for i in wave2] 
subj1 = [i.split("/")[-7] for i in wave1]

# remove duplicates 
subj4 = list(set(subj4))
subj3 = list(set(subj3))
subj2 = list(set(subj2))
subj1 = list(set(subj1))



global outpath
global DERIV_DIR
outpath = "/projects/niblab/bids_projects/Experiments/ChocoData/images"
DERIV_DIR = '/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'



In [ ]:
# SETTING UP IMAGE WITH ALLL 4 WAVES
# step one is to get all subject available in all waves
sub_list = [i for i in subj4 if i in (subj3 and subj2 and subj1)]
sub_list = sorted(sub_list)

filename = "exc_full_set.nii.gz"
all_funcs = []
for sub in sub_list:
    sub_funcs = glob.glob(os.path.join(DERIV_DIR, sub, "ses-*/func/Analysis/feat1/task*/filtered_func_data.nii.gz"))
    for y in sub_funcs:
        all_funcs.append(y)
all_funcs = sorted(all_funcs)
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in all_funcs)
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
outfile=os.path.join(outpath, filename)
#write the file
ni2_concat.to_filename(outfile)


##############################################
######### Set up the Behavioral CSV ##########
# Step one -get subjects

subj_dict = {}
for x in sub_list: 
    subj_dict[x] = [] 
    
# -add corresponding milkshake label    
ct=0
for d in subj_dict:
    task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
    for task in task_dirs:
        dir_name = task.split("/")[-1]
        sess = task.split("/")[-5]
        mlk = dir_name.split(".")[0].split("e")[1]
        id_ = sess+"_"+mlk
        subj_dict[d].append(id_)
        ct=ct+1
        
temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
for sub in subj_dict:
    file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
    for label in sorted(subj_dict[sub]):
        print(label.split("_")[1])
        fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
        fileX_contents = open(fileX, 'r')
        data = fileX_contents.read()
        fileX_contents.close()
        file.write(data)
    file.close()
    
tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))

# fill in 20% of test files 
test_slice_count = round(len(tempfiles) *.2)
test_slice = sub_list[:test_slice_count]
for sub in test_slice:
    df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
    df[1].replace(0,1, inplace=True)
    print(df.head())
    df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)    

# Make initial text file    
fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/exc_full_set.txt", "a")
fileout.write("Label sess\n")
       
for f in tempfiles:
    print("adding file %s"%f)
    f_contents = open(f, "r")
    data=f_contents.read()
    f_contents.close()
    fileout.write(data)
    
fileout.close()
# Make final CSV 
# MAKE CSV FILE WITH PANDAS
import pandas as pd
df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/exc_full_set.txt", sep=' ')
df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/exc_full_set.csv", sep='\t', index=False)




### Wave 4

#### Part 1

In [ ]:

# shuffle and divide subjects
shuffle(A_wave4)
B_wave4 = A_wave4[:len(A_wave4)//2]
C_wave4 = A_wave4[len(A_wave4)//2:]

# start building image
B_waves4_funcs = []
WAVE_01_filename = "4_waves_partA.nii.gz"

##############################################

for x in B_wave4:
    sess_func = glob.glob(os.path.join(DERIV_DIR, x, "ses-*/func/Analysis/feat1/task*/filtered_func_data.nii.gz"))
    for y in sess_func:
        B_waves4_funcs.append(y)
B_waves4_funcs = sorted(B_waves4_funcs)
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in B_waves4_funcs)
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
outfile=os.path.join(outpath,WAVE_01_filename)
#write the file
ni2_concat.to_filename(outfile)

##############################################
######### Set up the Behavioral CSV ##########
# Step one -get subjects
sub_list = B_wave4
subj_dict = {}
for x in sub_list: 
    subj_dict[x] = [] 
    
# -add corresponding milkshake label    
ct=0
for d in subj_dict:
    task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
    for task in task_dirs:
        dir_name = task.split("/")[-1]
        sess = task.split("/")[-5]
        mlk = dir_name.split(".")[0].split("e")[1]
        id_ = sess+"_"+mlk
        subj_dict[d].append(id_)
        ct=ct+1
        
temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
for sub in subj_dict:
    file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
    for label in sorted(subj_dict[sub]):
        print(label.split("_")[1])
        fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
        fileX_contents = open(fileX, 'r')
        data = fileX_contents.read()
        fileX_contents.close()
        file.write(data)
    file.close()
    
tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))

# fill in 20% of test files 
test_slice_count = round(len(tempfiles) *.2)
test_slice = sub_list[:test_slice_count]
for sub in test_slice:
    df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
    df[1].replace(0,1, inplace=True)
    print(df.head())
    df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)    

# Make initial text file    
fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/sub-001.txt", "a")
fileout.write("Label sess\n")
       
for f in tempfiles:
    print("adding file %s"%f)
    f_contents = open(f, "r")
    data=f_contents.read()
    f_contents.close()
    fileout.write(data)
    
fileout.close()
# Make final CSV 
# MAKE CSV FILE WITH PANDAS
import pandas as pd
df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/sub-001.txt", sep=' ')
df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/sub-001.csv", sep='\t', index=False)


#### Part 2

In [ ]:
C_waves4_funcs = []
WAVE4_02_filename = "4w_part2.nii.gz"
for x in C_wave4:
    sess_func = glob.glob(os.path.join(DERIV_DIR, x, "ses-*/func/Analysis/feat1/task*/filtered_func_data.nii.gz"))
    for y in sess_func:
        C_waves4_funcs.append(y)
C_waves4_funcs = sorted(C_waves4_funcs)
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in C_waves4_funcs)
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
outfile=os.path.join(outpath,WAVE4_02_filename)
#write the file
ni2_concat.to_filename(outfile)


##############################################
######### Set up the Behavioral CSV ##########
# Step one -get subjects
sub_list = C_wave4
subj_dict = {}
for x in sub_list: 
    subj_dict[x] = [] 
    
# -add corresponding milkshake label    
ct=0
for d in subj_dict:
    task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
    for task in task_dirs:
        dir_name = task.split("/")[-1]
        sess = task.split("/")[-5]
        mlk = dir_name.split(".")[0].split("e")[1]
        id_ = sess+"_"+mlk
        subj_dict[d].append(id_)
        ct=ct+1
        
temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
for sub in subj_dict:
    file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
    for label in sorted(subj_dict[sub]):
        fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
        fileX_contents = open(fileX, 'r')
        data = fileX_contents.read()
        fileX_contents.close()
        file.write(data)
    file.close()
    
# Make initial text file    
tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))
fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/02_4w_milkshake.txt", "a")
fileout.write("Label sess\n")
       
# fill in 20% of test files 
test_slice_count = round(len(tempfiles) *.2)
test_slice = sub_list[:test_slice_count]
for sub in test_slice:
    print(sub)
    df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
    df[1].replace(0,1, inplace=True)
    print(df.head())
    df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)
for f in tempfiles:
    print("adding file %s"%f)
    f_contents = open(f, "r")
    data=f_contents.read()
    f_contents.close()
    fileout.write(data)
    
fileout.close()
# WHEN READY - remove temp files
#os.remove(tempfiles)

# Make final CSV 
# MAKE CSV FILE WITH PANDAS
import pandas as pd
df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/02_4w_milkshake.txt", sep=' ')
df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w4_milkshake_B.csv", sep='\t', index=False)


## Batch waves 1-4: Segment B 